In [2]:
import csv
import json

# Define the path to your CSV file
csv_file_path = "D:\\Research\\AD\\ADLLM\\ad_train_nonhomo.csv"
# Define the path to the output JSON file
json_file_path = "D:\\Research\\AD\\ADLLM\\ad_train_nonhomo_serialization.json"
# Define the name of the label column (adjust it to your CSV)
label_column_name = 'AMYLOID_STATUS'  

#Records: The patient is a [Age] [PTMARRY] [PTGENDER] with [PTEDUCAT] years of education, a self-identified race [PTRACCAT], ethnicity [PTETHCAT], and is [PTMARRY]
#with [APOE4] APOE4 alleles. 
#The specimen results from Cerebrospinal Fluid (CSF) indicate that amyloid (Aβ) load is [ABETA] pg/mL, tau protein load is TAU pg/ML, and phosphorylated-tau load is [p-tau] pg/mL. 
#The average FDG-PET of angular, temporal, and posterior cingulate is [FDG]. 
#In cognitive testing, for clinical dementia rating (sum of boxes scores),  Alzheimer's Disease Assessment Scale 11 items, 13 items, question 4 score in the 11-item test, Mini-Mental State Exam,  the patients earned [CDRSB], [ADAS11], [ADAS13], [ADASQ4], and [MMSE], respectively.
#For the Rey Auditory Verbal Learning Test, the patient earned [RAVLT_immediate] for immediate, [RAVLT_learning] for learning, [RAVLT_forgetting] for forgetting, [RAVLT_perc_forgetting] for percent forgetting. 
#In Delayed Total Recall, Digit Span Test Score,  Trail Making Test Part B Time, Functional Assessment Questionnaire, Montreal Cognitive Assessment, Modified Preclinical Alzheimer Cognitive Composite with Digit test, 
#the patient earned [LDELTOTAL], [DIGITSCOR], [TRABSCOR], [FAQ], [MOCA], and [mpacc]. 
#In the Everyday Cognition test, the patient self-reported results for memory, Language, Visuospatial Abilities, Planning, Organization, Divided attention, and  Total scores are  [EcogSPMem], [EcogSPLang], [EcogSPVisspat], [EcogSPPlan], [EcogSPOrgan], [EcogSPDivatt], [EcogSPTotal], while the partner-reported results are  [EcogPtMem], [EcogPtLang], [EcogPtVisspat], [EcogPtPlan], [EcogPtOrgan], [EcogPtDivatt], [EcogPtTotal]. 
#The MRI image also indicates that the volume of this patient’s Ventricles, Hippocampus, WholeBrain, Entorhinal, Fusiform, middle temporal area, and intracranial are [Ventricles], [Hippocampus], [WholeBrain], [Entorhinal], [Fusiform], [Midtemp], and [ICV]. 
#The overall diagnosis status is [DX].



# Function to convert a row from the CSV to a markdown table
def row_to_markdown(row):
    prompt= "Classify the following patient into one of the above classes. Please answer in a single word. Patient's record:"
    prompt_age_sex_race="The patient is a "+row['Age']+" year-old "+row['PTGENDER']+" and the marital status is "+row['PTMARRY']+", with "+ row['PTEDUCAT']+" years of education, a self-identified race "+row["PTRACCAT"]+", ethinicity " +row["PTETHCAT"] #no missing
    if row['APOE4']=="NA":
        prompt_gene=". "
    elif int(row['APOE4'])==0:
        prompt_gene=", with no APOE4 allele. "
    elif int(row['APOE4'])==1:
        prompt_gene=", with  one APOE4 allele. "
    elif int(row['APOE4'])==2:
        prompt_gene=", with two APOE4 alleles. "
        
    if row["ABETA"]=="NA":
        prompt_specimen=""
    else:
        prompt_specimen="The specimen results from Cerebrospinal Fluid (CSF) indicate that beta-amyloid load is "+row["ABETA"]+ "pg/mL, tau protein load is "+row["TAU"]+" pg/mL, and phosphorylated-tau load is "+row["PTAU"]+" pg/mL."
    if row["FDG"]=='NA':
        prompt_FDG=""
    else:
        prompt_FDG="The average FDG-PET of angular, temporal, and posterior cingulate is "+row["FDG"]+". "
    test_name=" In cognitive testing, for "
    test_score=""
    if row['CDRSB']!="NA":
        test_name=test_name+"clinical dementia rating (sum of boxes scores), "
        test_score=row['CDRSB']
    if row['ADAS11']!="NA":
        test_name=test_name+"Alzheimer's Disease Assessment Scale (11 items), "
        test_score=test_score+", "+ row["ADAS11"]
    if row['ADAS13']!="NA":
        test_name=test_name+"Alzheimer's Disease Assessment Scale (13 items), "
        test_score=test_score+", "+row["ADAS13"]
    if row['ADASQ4']!="NA":
        test_name=test_name+"question 4 score in the 11-item test, "
        test_score=test_score+", "+row["ADASQ4"]
    if row['MMSE']!="NA":
        test_name=test_name+"Mini-Mental State Exam, "
        test_score=test_score+", "+row['MMSE']
    prompt_cognitive_test=test_name+"the patients earned "+test_score+", respectively. "
    if row["RAVLT_immediate"]=="NA":
        prompt_ravlt=""
    else:
        prompt_ravlt="For the Rey Auditory Verbal Learning Test, the patient earned "+ row["RAVLT_immediate"]+ " for immediate, "+row["RAVLT_learning"]+ " for learning, "+ row["RAVLT_forgetting"]+ " for forgetting, " +row["RAVLT_perc_forgetting"] +" for percent forgetting. "

    test_name="In "
    test_score=""
    if row['LDELTOTAL']!="NA":
        test_name=test_name+"Delayed Total Recall test, "
        test_score=test_score+row['LDELTOTAL']
    if row['DIGITSCOR']!="NA":
        test_name=test_name+"Digit Span test, "
        test_score=test_score+", "+ row["DIGITSCOR"]
    if row['TRABSCOR']!="NA":
        test_name=test_name+"Trail Making Test Part B Time, "
        test_score=test_score+", "+row["TRABSCOR"]
    if row['FAQ']!="NA":
        test_name=test_name+"Functional Assessment Questionnaire, "
        test_score=test_score+", "+row["FAQ"]
    if row['MOCA']!="NA":
        test_name=test_name+"Montreal Cognitive Assessment, "
        test_score=test_score+", "+row['MOCA']
    if row['mPACCdigit']!="NA":
        test_name=test_name+" Modified Preclinical Alzheimer Cognitive Composite with Digit test, "
        test_score=test_score+", "+row['mPACCdigit']
    prompt_cognitive_test2=test_name+"the patients earned "+test_score+", respectively. "
    test_name="In the Everyday Cognition test, the patient self-reported results for memory, Language, Visuospatial Abilities, Planning, Organization, Divided attention, and  Total scores are "
    test_score=""
    if row["EcogSPMem"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogSPMem"]+", "
    if row["EcogSPLang"]=="NA":
        test_score=test_score+"unknown,"
    else:
        test_score=test_score+row["EcogSPLang"]+", "
    if row["EcogSPVisspat"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogSPVisspat"]+", "
    if row["EcogSPPlan"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogSPPlan"]+", "
    if row["EcogSPOrgan"]=="NA":
        test_score=test_score+"unknown,"
    else:
        test_score=test_score+row["EcogSPOrgan"]+", "
    if row["EcogSPDivatt"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogSPDivatt"]+", "
    
    if row["EcogSPTotal"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogSPTotal"]+", "
    prompt_everyday_sp=test_name+test_score
    prompt_trans="while the partner-reported results are "
    test_score=""
    if row["EcogPtMem"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogPtMem"]+", "
    if row["EcogPtLang"]=="NA":
        test_score=test_score+"unknown,"
    else:
        test_score=test_score+row["EcogPtLang"]+", "
    if row["EcogPtVisspat"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogPtVisspat"]+", "
    if row["EcogPtPlan"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogPtPlan"]+", "
    if row["EcogPtOrgan"]=="NA":
        test_score=test_score+"unknown,"
    else:
        test_score=test_score+row["EcogPtOrgan"]+", "
    if row["EcogPtDivatt"]=="NA":
        test_score=test_score+"unknown, "
    else:
        test_score=test_score+row["EcogPtDivatt"]+", "
    
    if row["EcogPtTotal"]=="NA":
        test_score=test_score+"unknown."
    else:
        test_score=test_score+row["EcogPtTotal"]+". "
    prompt_everyday_pt=prompt_trans+test_score
    if row["Ventricles"]=="NA" and row["Hippocampus"]=="NA" and row["WholeBrain"]=="NA" and row["Entorhinal"]=="NA" and row["Fusiform"]=="NA" and row["MidTemp"]=="NA" and row["ICV"]=="NA":
        prompt_volume=""
    test_name="The MRI image also indicates that the volume of this patient's "
    test_score=""
    if row["Ventricles"]!="NA":
        test_name=test_name+"Ventricles, "
        test_score=test_score+", "+row["Ventricles"]
    if row["Hippocampus"]!="NA":
        test_name=test_name+"Hippocampus, "
        test_score=test_score+", "+row["Hippocampus"]
    if row["WholeBrain"]!="NA":
        test_name=test_name+"WholeBrain, "
        test_score=test_score+", "+row["WholeBrain"]
    if row["Entorhinal"]!="NA":
        test_name=test_name+"Entorhinal, "
        test_score=test_score+", "+row["Entorhinal"]    
    if row["Fusiform"]!="NA":
        test_name=test_name+"Fusiform, "
        test_score=test_score+", "+row["Fusiform"]
    if row["MidTemp"]!="NA":
        test_name=test_name+"middle temporal area, "
        test_score=test_score+", "+row["MidTemp"]
    if row["ICV"]!="NA":
        test_name=test_name+"intracranial, "
        test_score=test_score+", "+row["ICV"]
    prompt_volume=test_name+" are "+test_score+"."
    if row["DX"]=="NA":
        prompt_dx=""
    elif row["DX"]=="CN":
        prompt_dx=" The overall diagnosis status for the patient is cognitively normal."
    elif row["DX"]=="Dementia":
        prompt_dx=" The overall diagnosis status for the patient is dementia."
    elif row["DX"]=="MCI":
        prompt_dx=" The overall diagnosis status for the patient is mildly cognitive impairment."
    
    prompt_final=prompt+prompt_age_sex_race+prompt_gene+prompt_specimen+ prompt_FDG+prompt_cognitive_test+prompt_ravlt+prompt_cognitive_test2+prompt_everyday_sp+prompt_everyday_pt+prompt_volume+prompt_dx
    return prompt_final

# Function to convert CSV to JSON format with the specified structure
def csv_to_json(csv_filepath, json_filepath, label_col_name):
    data_for_json = []
    with open(csv_filepath, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            label = row.pop(label_col_name, None)
            if label is None:
                continue  # Skip rows where the label is missing
            
            # Convert the row to a markdown table
            input_table = row_to_markdown(row)
            #print(input_table)
            # Determine the response based on the label, make sure to customize this to your use case
            response = 'Yes' if int(label) == 1 else 'No'
            #added_label = 'Positive.' if label == '1' else 'Negative.'
            #Create the JSON entry
            entry = {
                "system": "As a Question Answering agent, your goal is to predict whether the input patient will have a positive beta amyloid-PET (Positron emission tomography) scan based on their health records, including demographic characteristics, genetic information, blood test results, MRI neuroimaging results, and cognitive testing scores.  So that the prediction can be utilized in later stages of diagnosis. Following are the classes of whether having a positive beta amyloid-PET scan [Yes, No]. Pay attention to the number of APOE4 alleles, beta-amyloid load load in CSF, and cognitive testing scores.",
                "user":f"{input_table}",
                "completion": f"{response}"
            }
            
            data_for_json.append(entry)
    
    # Write the data to the JSON file
    with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
        json.dump(data_for_json, jsonfile, indent=4, ensure_ascii=False)

csv_to_json(csv_file_path, json_file_path, label_column_name)
print("JSON file has been created successfully.")



JSON file has been created successfully.


In [1]:
import pandas as pd
abeta_amyloid=pd.read_csv("D:\\Research\\AD\\ADLLM\\UCBERKELEY_AMY_6MM_14Apr2024.csv")

In [2]:
predictors=pd.read_csv("D:\\Research\\AD\\ADLLM\\ADNI_BASELINE_48_predictors.csv")

In [5]:
filtered_abeta_amyloid = abeta_amyloid[(abeta_amyloid['RID'].isin(predictors['RID'])) & 
                                        (abeta_amyloid['VISCODE'].isin(['bl', 'init']))]

In [7]:
filtered_abeta_amyloid.to_csv("D:\\Research\\AD\\ADLLM\\filtered_abeta_amyloid.csv", index=False)

In [12]:
# Select the RID and abeta_status columns from the filtered dataframe
abeta_status = filtered_abeta_amyloid[['RID', 'AMYLOID_STATUS']]

# Merge the abeta_status with predictors dataframe on RID
merged_data = predictors.merge(abeta_status, on='RID', how='right')

In [13]:
merged_data 

,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,AGE_BL,PTGENDER,PTEDUCAT,...,MidTemp,ICV,DX,mPACCdigit,Years_bl,Month_bl,Month,M,Age,AMYLOID_STATUS
0,767,ADNI1,ADNI1,141_S_0767,141,bl,2007/9/6,73.0,Female,16,...,18266.0,1341320.0,CN,-0.792259,0,0,0,0,73.0,0.0
1,4214,ADNI2,ADNI2,037_S_4214,37,bl,2003/10/11,74.2,Male,20,...,18121.0,1499670.0,MCI,-1.941270,0,0,0,0,74.2,0.0
2,4856,ADNI2,ADNI2,070_S_4856,70,bl,2003/8/12,65.0,Female,18,...,20221.0,1490290.0,CN,3.827690,0,0,0,0,65.0,0.0
3,4944,ADNI2,ADNI2,052_S_4944,52,bl,2009/1/13,68.0,Male,14,...,20642.0,1354850.0,MCI,-5.332670,0,0,0,0,68.0,1.0
4,6001,ADNI3,ADNI3,027_S_6001,27,bl,2019/1/17,72.0,Female,14,...,16957.0,1169670.0,CN,-0.706072,0,0,0,0,72.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,7089,ADNI3,ADNI3,011_S_7089,11,bl,2003/8/22,65.7,Female,16,...,NaN,NaN,CN,-0.341944,0,0,0,0,65.7,NaN
1053,7092,ADNI3,ADNI3,021_S_7092,21,bl,2012/7/22,68.5,Male,15,...,20522.0,1577290.0,CN,-0.341944,0,0,0,0,68.5,0.0
1054,7105,ADNI3,ADNI3,035_S_7105,35,bl,2020/10/22,77.5,Male,18,...,NaN,NaN,MCI,-14.350300,0,0,0,0,77.5,1.0
1055,7117,ADNI3,ADNI3,082_S_7117,82,bl,2012/1/23,68.9,Female,18,...,NaN,NaN,MCI,2.742450,0,0,0,0,68.9,0.0


In [14]:
filtered_merged_data = merged_data.dropna(subset=['AMYLOID_STATUS'])

In [30]:
filtered_merged_data = filtered_merged_data.fillna('NA')

In [32]:
filtered_merged_data.to_csv("D:\\Research\\AD\\ADLLM\\filtered_abeta_amyloid_merged.csv", index=False)

In [34]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(filtered_merged_data, test_size=0.2, random_state=42)

In [35]:
filtered_merged_data

,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,AGE_BL,PTGENDER,PTEDUCAT,...,MidTemp,ICV,DX,mPACCdigit,Years_bl,Month_bl,Month,M,Age,AMYLOID_STATUS
0,767,ADNI1,ADNI1,141_S_0767,141,bl,2007/9/6,73.0,Female,16,...,18266.0,1341320.0,CN,-0.792259,0,0,0,0,73.0,0.0
1,4214,ADNI2,ADNI2,037_S_4214,37,bl,2003/10/11,74.2,Male,20,...,18121.0,1499670.0,MCI,-1.94127,0,0,0,0,74.2,0.0
2,4856,ADNI2,ADNI2,070_S_4856,70,bl,2003/8/12,65.0,Female,18,...,20221.0,1490290.0,CN,3.82769,0,0,0,0,65.0,0.0
3,4944,ADNI2,ADNI2,052_S_4944,52,bl,2009/1/13,68.0,Male,14,...,20642.0,1354850.0,MCI,-5.33267,0,0,0,0,68.0,1.0
4,6001,ADNI3,ADNI3,027_S_6001,27,bl,2019/1/17,72.0,Female,14,...,16957.0,1169670.0,CN,-0.706072,0,0,0,0,72.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,7076,ADNI3,ADNI3,035_S_7076,35,bl,2028/9/22,62.8,Female,16,...,NA,NA,CN,1.88567,0,0,0,0,62.8,0.0
1053,7092,ADNI3,ADNI3,021_S_7092,21,bl,2012/7/22,68.5,Male,15,...,20522.0,1577290.0,CN,-0.341944,0,0,0,0,68.5,0.0
1054,7105,ADNI3,ADNI3,035_S_7105,35,bl,2020/10/22,77.5,Male,18,...,NA,NA,MCI,-14.3503,0,0,0,0,77.5,1.0
1055,7117,ADNI3,ADNI3,082_S_7117,82,bl,2012/1/23,68.9,Female,18,...,NA,NA,MCI,2.74245,0,0,0,0,68.9,0.0


In [38]:
train_data.to_csv("D:\\Research\\AD\\ADLLM\\ad_train_data.csv", index=False)
test_data.to_csv("D:\\Research\\AD\\ADLLM\\ad_test_data.csv", index=False)

In [36]:
train_data

,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,AGE_BL,PTGENDER,PTEDUCAT,...,MidTemp,ICV,DX,mPACCdigit,Years_bl,Month_bl,Month,M,Age,AMYLOID_STATUS
395,337,ADNI1,ADNI1,021_S_0337,21,bl,2025/4/6,75.8,Male,20,...,22773.0,1817900.0,CN,0.887969,0,0,0,0,75.8,0.0
257,6724,ADNI3,ADNI3,126_S_6724,126,bl,2011/6/19,75.4,Male,17,...,19759.0,1571230.0,MCI,-10.3091,0,0,0,0,75.4,1.0
358,7029,ADNI3,ADNI3,067_S_7029,67,bl,2011/2/22,59.8,Female,14,...,20424.0,1396160.0,MCI,-1.99839,0,0,0,0,59.8,0.0
60,6183,ADNI3,ADNI3,027_S_6183,27,bl,2008/2/18,65.6,Male,16,...,22741.0,1423100.0,CN,1.14313,0,0,0,0,65.6,0.0
349,7011,ADNI3,ADNI3,037_S_7011,37,bl,2022/12/21,50.4,Female,17,...,21283.0,1328600.0,CN,-2.41964,0,0,0,0,50.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,6254,ADNI3,ADNI3,941_S_6254,941,bl,2022/5/19,68.7,Male,14,...,19196.0,1370420.0,CN,-2.72665,0,0,0,0,68.7,1.0
333,6966,ADNI3,ADNI3,070_S_6966,70,bl,2012/7/21,61.5,Male,20,...,21625.0,1453900.0,CN,2.2855,0,0,0,0,61.5,0.0
473,2121,ADNIGO,ADNIGO,082_S_2121,82,bl,2003/11/10,67.8,Female,14,...,22623.0,1584900.0,MCI,-8.94542,0,0,0,0,67.8,1.0
121,6370,ADNI3,ADNI3,027_S_6370,27,bl,2023/7/18,69.8,Female,18,...,NA,1593690.0,MCI,-4.7258,0,0,0,0,69.8,1.0
